In [1]:
import PyPDF2

import re

import nltk
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords

from collections import OrderedDict
import pprint

import pandas as pd
import numpy as np

import os

import tensorflow as tf
import tensorflow_hub as hub

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

nltk.download('punkt')

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
keywords_df = pd.read_csv("keywords.csv")
keywords_list = list(keywords_df["Sustainability Keywords"])

target_keywords_df = pd.read_csv("target_keywords.csv")
target_keywords_list = list(target_keywords_df["Sustainability Keywords"])

def read_in_pdf(input_name):
    pdffileobj=open(input_name,'rb')
    pdfreader=PyPDF2.PdfFileReader(pdffileobj, strict=False)
    x=pdfreader.numPages
    try:
        os.remove("intermediate_file1.txt")
    except OSError:
        pass
    for i in range(x):
        pageobj=pdfreader.getPage(i)
        text=pageobj.extractText()

        file1=open(r"intermediate_file1.txt","a", encoding="utf-8")
        file1.writelines(text)
        file1.close()
    f=open(r"intermediate_file1.txt","r", encoding="utf-8")
    temp = f.read().splitlines()
    text = ' '.join(temp)
    return text


def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))


def process_company(keywords_list, report_directory_in_str, company_folder_in_str, target_reports_l, sim_threshold):
    report_directory_in_str += "/"
    company_folder_in_str += "/"
    files_directory_in_str = report_directory_in_str + company_folder_in_str
    file_directory = os.fsencode(files_directory_in_str)

    output_df = pd.DataFrame()
    for keyword in keywords_list:
        print("Processing: " + keyword)
        part_df = pd.DataFrame()
        for file in os.listdir(file_directory):
            report_col_df = pd.DataFrame()
            filename = os.fsdecode(file)
            if (str(filename) in target_reports_l) and filename.endswith(".pdf"):
                sentence_list = []
                print("Processing: " + str(filename))
                text = read_in_pdf(report_directory_in_str + company_folder_in_str + str(filename))
                sentences = nltk.Text(sent_tokenize(text))
                for sent in sentences:
                    sim = cosine(model([keyword])[0], model([sent])[0])
                    if len(sent.split()) > 4 and sim > sim_threshold:
                        sentence_list.append(sent)
                temp_dict = dict()
                temp_dict[filename] = sentence_list
                temp_col_df = pd.DataFrame.from_dict(temp_dict)
                part_df = pd.concat([part_df, temp_col_df], axis=1)
        part_df["keyword"] = [keyword] * len(part_df)
        output_df = pd.concat([output_df, part_df], axis=0)
    return output_df

In [6]:
output_Coty = process_company(target_keywords_list, "Reports", "Coty", ["coty_sustainability_report_fy20.pdf#pa.pdf"], 0.2)
display(output_Coty)
output_Coty.to_csv("output_Coty.csv", index=False)

Processing: biodegradable
Processing: coty_sustainability_report_fy20.pdf#pa.pdf
Processing: emissions
Processing: coty_sustainability_report_fy20.pdf#pa.pdf
Processing: water
Processing: coty_sustainability_report_fy20.pdf#pa.pdf


,coty_sustainability_report_fy20.pdf#pa.pdf,keyword
0,BEAUTY THAT LASTS: SUSTAINABILITY REPORT 20...,biodegradable
1,One example is the COVERGIRL Clean Fresh range...,biodegradable
2,"As new challenges emerge, we will continue to...",biodegradable
3,"INTRODUCTION Throughout our own operations, we...",biodegradable
4,We are doing this by committing to reduce our...,biodegradable
...,...,...
0,To build an understanding of our impact relati...,water
1,Water is a precious planetary resource that s...,water
2,We have a responsibility to reduce our water ...,water
3,To help us identify the areas within our busi...,water


In [7]:
output_estee_lauder = process_company(target_keywords_list, "Reports", "estee lauder", ["elc-fy20-citizenship-sustainability-report.pdf"], 0.2)
display(output_estee_lauder)
output_estee_lauder.to_csv("output_estee_lauder.csv", index=False)

Processing: biodegradable
Processing: elc-fy20-citizenship-sustainability-report.pdf
Processing: emissions
Processing: elc-fy20-citizenship-sustainability-report.pdf
Processing: water
Processing: elc-fy20-citizenship-sustainability-report.pdf


,elc-fy20-citizenship-sustainability-report.pdf,keyword
0,Additional information is available in our ˜...,biodegradable
1,Industrial waste includes the solid componen...,biodegradable
2,"PACKAGING By 2025, 75-100% of our packaging w...",biodegradable
3,55% of our packaging by weight is either recyc...,biodegradable
4,We will increase the amount of post-consumer ...,biodegradable
...,...,...
5,Aveda™s goal is to double its impact with ch...,water
6,"ELC™s Melville, New York, facilities are locat...",water
7,"3-4 303-4 Water discharge Metrics, p. 105 (Wa...",water
8,"3-5 303-5 Water consumption Metrics, p. 105 (W...",water


In [8]:
output_jnj = process_company(target_keywords_list, "Reports", "jnj", ["2015-JNJ-Citizenship-Sustainability-Report.pdf"], 0.2)
display(output_jnj)
output_jnj.to_csv("output_jnj.csv", index=False)

Processing: biodegradable
Processing: 2015-JNJ-Citizenship-Sustainability-Report.pdf
Processing: emissions
Processing: 2015-JNJ-Citizenship-Sustainability-Report.pdf
Processing: water
Processing: 2015-JNJ-Citizenship-Sustainability-Report.pdf


,2015-JNJ-Citizenship-Sustainability-Report.pdf,keyword
0,We must maintain in good order the property w...,biodegradable
1,Product Stewardship Improving the sustainabili...,biodegradable
2,"Using our Earthwards ® approach, we are reduci...",biodegradable
3,Reduce absolute carbon emissions 20 percent ...,biodegradable
4,Produce/procure 20 percent of electricity fro...,biodegradable
...,...,...
26,Examples of water conservation efforts are hig...,water
27,)Aspect: Water G4-DMAGeneric Disclosures on Ma...,water
28,Water impacts are managed locally.,water
29,"G4-EN26Identity, size, protected status, and b...",water


In [9]:
output_loreal = process_company(target_keywords_list, "Reports", "loreal", ["2019-progress-report.pdf"], 0.3)
display(output_loreal)
output_loreal.to_csv("output_loreal.csv", index=False)

Processing: biodegradable
Processing: 2019-progress-report.pdf
Processing: emissions
Processing: 2019-progress-report.pdf
Processing: water
Processing: 2019-progress-report.pdf


,2019-progress-report.pdf,keyword
0,Strengthening L’Oréal’s sustainable sourcing...,biodegradable
1,"In particular, L’Oréal has committed to maki...",biodegradable
2,"To promote the circular economy, Biotherm has...",biodegradable
0,-78% reduction in CO 2 emissions from plants a...,emissions
1,13PROGRESS REPORT 2019MONITORING INDICATORS UP...,emissions
2,These values take into account the emissions c...,emissions
3,"Between 2005 and 2019, L’Oréal reduced its GHG...",emissions
4,ASIA PACIFIC AND AMERICAS WORKSHOPS TO ENGAGE ...,emissions
5,This ambition reinforces L’Oréal’s low carbo...,emissions
0,These studies highlight that water is one of t...,water


In [10]:
output_png = process_company(target_keywords_list, "Reports", "p&g", ["citizenship_report_2020_full.pdf"], 0.3)
display(output_png)
output_png.to_csv("output_png.csv", index=False)

Processing: biodegradable
Processing: citizenship_report_2020_full.pdf
Processing: emissions
Processing: citizenship_report_2020_full.pdf
Processing: water
Processing: citizenship_report_2020_full.pdf


,citizenship_report_2020_full.pdf,keyword
0,Ł 90% of product packaging is either recyclabl...,biodegradable
1,"To further enable responsible consumption, w...",biodegradable
2,"Today, almost all Always packaging is recyclab...",biodegradable
3,"2 Today, all of our diaper packaging is recycl...",biodegradable
4,Brand 2030 AIM NUDGING FOR GOOD AWARD RECYCL...,biodegradable
5,Charmin continues to use recycled ˜bers wher...,biodegradable
6,To further enable and inspire responsible cons...,biodegradable
7,We have also clearly de˜ned goals to decrease...,biodegradable
8,We will continue to report progress versus ou...,biodegradable
9,Ł 100% of our packaging will be recyclable or ...,biodegradable


In [11]:
output_shiseido = process_company(target_keywords_list, "Reports", "shiseido", ["2019.pdf"], 0.3)
display(output_shiseido)
output_shiseido.to_csv("output_shiseido.csv", index=False)

Processing: biodegradable
Processing: 2019.pdf
Processing: emissions
Processing: 2019.pdf
Processing: water
Processing: 2019.pdf


,2019.pdf,keyword
0,"Complying with 5Rs (Respect, Reduce, Reuse, Re...",biodegradable
1,sustainable packaging Replace Sustainable an...,biodegradable
2,Reuse also helps to stop single-use plastic us...,biodegradable
3,In order to make it easy to recycle the used c...,biodegradable
4,"However, while recycling is really im- Produce...",biodegradable
0,"To help combat climate change, we are stepping...",emissions
1,"As a result, energy consumption as well as CO...",emissions
0,"Water Across the Shiseido Group, we aim to ens...",water


In [12]:
output_unilever = process_company(target_keywords_list, "Reports", "unilever", ["uslp-3-year-performance-summary-2017-2019_tcm244-549781_en.pdf"], 0.3)
display(output_unilever)
output_unilever.to_csv("output_unilever.csv", index=False)

Processing: biodegradable
Processing: uslp-3-year-performance-summary-2017-2019_tcm244-549781_en.pdf
Processing: emissions
Processing: uslp-3-year-performance-summary-2017-2019_tcm244-549781_en.pdf
Processing: water
Processing: uslp-3-year-performance-summary-2017-2019_tcm244-549781_en.pdf


,uslp-3-year-performance-summary-2017-2019_tcm244-549781_en.pdf,keyword
